In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

os.makedirs('utils/', exist_ok=True)
os.chdir('utils')

! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/models.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/losses.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/vision.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/callbacks.py
! wget -q https://raw.githubusercontent.com/Ata-Pab/Machine_Learning/master/utils/utils.py

os.chdir('/content')
print("Current working directory", os.getcwd())

Current working directory /content


In [4]:
from utils import vision
from utils import utils
from utils import losses

### Experiment Setup Start

In [36]:
# Create experiment folder
EXPERIMENT_NAME = '...'
EXPERIMENT_SAVE_DIR = os.path.join('/content/drive/MyDrive/MASTER/Master_Thesis/Experiments', EXPERIMENT_NAME)

In [35]:
experiment = {
    'TYPE': 'test',        # Experiment type: 'train', 'test'
    'ACCELERATOR': 'GPU',   # 'CPU', 'GPU' or 'TPU'

    # Input data
    'DATASET': 'BD67_Dataset',
    'IMAGE_SIZE': (256, 256),
    'INPUT_SHAPE': (256, 256, 3),
    'VALID_SIZE': 0.1,      # Validation data size: (Valid Data) / (All Data)
    'DATA_AUG': True,       # Apply data augmentation
    'DATA_AUG_POWER': 1,    # Data augmentation power: How many times data
     # augmentation will be applied to the whole dataset. default 1

    # Model
    'BACKBONE': 'custom',        # 'custom', 'VGG16', 'VGG19' - default 'custom'
    'LAST_TRANIABLE_LAYERS': 5,  # number of last trainable layers of pre-trained backbone models, fine-tuning
    'BATCH_SIZE': 16,            # IF TPU is active set 4, otherwise set anything
    'EPOCHS': 2,
    'OPTIMIZER': 'Adam',  # TODO: Try 'rmsprop' optimizer
    'LEARNING_RATE': 1e-4,
    # set latent dim - shape: (LATENT_DIM, 1) - default 200
    'LATENT_DIM': 500,

    # Loss
    'RECONS_LOSS': 'PERCEPTUAL',  # Reconstruction loss ('SSIM', 'MSE', 'MAE', 'PERCEPTUAL')
    'PERCEPTUAL_LAYERS': [5,8,13,18],    # [5,8,13,18], None
    'PERCEP_LOSS_MODEL': 'VGG19', # 'custom', 'VGG16', 'VGG19', 'ResNet50' - default 'VGG16'
    'PERP_LOSS_LAMBDA': 1,
    'LRELU_SLOPE': 0.2,       # Leaky ReLU activation function slope value
    'MSE_LOSS_LAMBDA': 0.01,  # MSE coeff

    # Save model
    'SAVE_WEIGHTS_PER_EPOCH': 1,  # Checkpoints
}

In [37]:
if experiment['TYPE'] == 'train':
    assert(EXPERIMENT_NAME != '...')
    # Create experiment folder
    os.makedirs(EXPERIMENT_SAVE_DIR, exist_ok=True)

    # Model checkpoints will be save in exp_save_dir
    exp_save_dir = utils.create_experimental_output(experiment, EXPERIMENT_SAVE_DIR)

    TRAINING_WEIGHT_DIR = os.path.join(exp_save_dir, 'training_weights')
    # Create folder for checkpoints (training weights)
    os.makedirs(TRAINING_WEIGHT_DIR, exist_ok=True)
else:  # test mode
    # Set experiment save directory and training weight directory manually
    exp_save_dir = '/content/drive/MyDrive/MASTER/Master_Thesis/Experiments/SiamDecoderResNet50Model/experiment_1'
    TRAINING_WEIGHT_DIR = os.path.join(exp_save_dir, 'training_weights')

In [38]:
print(f"...Experiment {exp_save_dir.split('experiment_')[1]} was initialized...")
print(f"Experiment directory: {EXPERIMENT_SAVE_DIR}")
print(f"Training weights save directory: {TRAINING_WEIGHT_DIR}")

...Experiment 1 was initialized...
Experiment directory: /content/drive/MyDrive/MASTER/Master_Thesis/Experiments/...
Training weights save directory: /content/drive/MyDrive/MASTER/Master_Thesis/Experiments/SiamDecoderResNet50Model/experiment_1/training_weights


### Experiment Setup End

### Build Custom model

In [22]:
class CustomModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """
    def __init__(self, input_shape):
        ...
        return 0

In [ ]:
def debug_training_process(model, epoch_num, data_input, metrics):
  ...

  plt.show()

In [25]:
if experiment['ACCELERATOR'] != 'TPU':
  @tf.function
  def train_step(images):
      ...

      return 0

In [26]:
from IPython import display
import time

if experiment['ACCELERATOR'] != 'TPU':
  def train(dataset, epochs):
      loss_hist = []  # Keep loss history
      for epoch in range(epochs):
          start = time.time()
          for image_batch in dataset:
              loss = train_step(image_batch)

          loss_hist.append(loss)   # Add loss value to the loss history after each epoch
          print("loss: ", loss)

          # Save the model every experiment['SAVE_WEIGHTS_PER_EPOCH'] epochs
          if (epoch + 1) % experiment['SAVE_WEIGHTS_PER_EPOCH'] == 0:
            seed = image_batch[:experiment['BATCH_SIZE']]
            display.clear_output(wait=True)
            debug_training_process(siamese_resnet50_model,
                                      epoch + 1,
                                      seed,
                                      loss)

            # Save checkpoints
            utils.save_experiment_checkpoints([...], epoch=(epoch+1), save_dir=TRAINING_WEIGHT_DIR)
            print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

      # Generate after the final epoch
      display.clear_output(wait=True)
      debug_training_process(siamese_resnet50_model,
                              epochs,
                              seed,
                              loss)
      return loss_hist

In [28]:
if experiment['TYPE'] == 'train':
    custom_model.compile(ae_optimizer=tf.keras.optimizers.Adam(0.0001), siam_optimizer=tf.keras.optimizers.Adam(0.0001))
    #custom_model.fit(..., epochs=experiment['EPOCHS'], validation_data=...)
    custom_model_hist = train(..., experiment['EPOCHS'])
else:  # test mode
    # Set load weight epoch number manually
    utils.load_model_experiment_weights([...], epoch=..., load_dir=TRAINING_WEIGHT_DIR)